#             ***Understanding the problem***

Autonomous vehicles (AVs) are expected to dramatically redefine the future of transportation. However, there are still significant engineering challenges to be solved before one can fully realize the benefits of self-driving cars. One such challenge is building models that reliably predict the movement of traffic agents around the AV, such as cars, cyclists, and pedestrians.

The ridesharing company Lyft started Level 5 to take on the self-driving challenge and build a full self-driving system. Their previous competition tasked participants with identifying 3D objects, an important step prior to detecting their movement. Now, they’re challenging to predict the motion of these traffic agents.

So we are trying to train motion prediction models with the largest collection of prediction data released to date. The dataset includes the logs of over 1,000 hours of movement of various traffic agents—such as cars, cyclists, and pedestrians—that is captured by autonomous fleet encountered on Palo Alto routes.

This articles gives you more understanding the problem

https://medium.com/lyftlevel5/fueling-self-driving-research-with-level-5s-open-prediction-dataset-f0175e2b0cf8


## The Road Map to Autonomus Driving is

In [ ]:
from IPython.display import Image
import os


Image("../input/images-lyft/01.JPG")

## Problem we are trying to solving

In [ ]:
Image("../input/images-lyft/02.JPG")

## Data Format

* The dataset is provided in zarr format. The zarr files are flat, compact, and highly performant for loading.

* The dataset consists of frames and agent states. A frame is a snapshot in time which consists of ego pose, time, and multiple agent states. Each agent state describes the position, orientation, bounds, and type.
   
 
Documentation of the library created by Level 5 to read the dataset

1. http://www.l5kit.org/
2. https://github.com/lyft/l5kit/blob/master/examples/visualisation/visualise_data.ipynb

In [ ]:
DIR_PATH = '../input/lyft-motion-prediction-autonomous-vehicles/'
os.listdir(DIR_PATH)

In [ ]:
os.listdir('../input/lyft-motion-prediction-autonomous-vehicles/scenes')

## Importing required libraries 

In [ ]:
# !pip install --upgrade pip
# !pip install pymap3d==2.1.0
# !pip install -U l5kit

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset

from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from tqdm import tqdm
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable

By setting the L5KIT_DATA_FOLDER variable, we can point the script to the folder where the data lies

In [ ]:
# set env variable for data
import os
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"

# reading the data
zarr_dataset = ChunkedDataset('../input/lyft-motion-prediction-autonomous-vehicles/scenes/train.zarr')
zarr_dataset.open()
print(zarr_dataset)

.zarr files support most of the traditional numpy array operations. In the following cell we iterate over the frames to get a scatter plot of the AV locations to understand the distributions of various agent type distribution.

In [ ]:
cfg={
    
    'model_params':{
        'model_architecture': "resnet50",
        'history_num_frames': 0,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
        
    },

    
    
    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },
    
    'test_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 8,
        'shuffle': False,
        'num_workers': 4
    }
}

In [ ]:
dm = LocalDataManager(None)
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)

## Visualize the Autonomous vehicles

In [ ]:
data = dataset[1]

im = data["image"].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

plt.imshow(im[::-1])
plt.show()

## Satilite Imagary

In [ ]:
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
data = dataset[1]

im = data["image"].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

plt.imshow(im[::-1])
plt.show()

## Visualise the agent

In [ ]:
dataset = AgentDataset(cfg, zarr_dataset, rast)
data = dataset[1]

im = data["image"].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

plt.imshow(im[::-1])
plt.show()

## Entire picture

In [ ]:
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_semantic"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

# Build model for motion prediction

WORK IN PROGRESS


### <span style="color:red">*If you find this kernel useful, Please consider Upvoting it , it motivates me to write more content.* </span>
